In [10]:
sheet_id='replace with your sheets id'
sheet_name='Replace with the sheet name'

In [2]:
import pandas as pd

import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

# 1. Load your service account credentials
SERVICE_ACCOUNT_FILE = r'Path to the service account'

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# 2. Connect to Google Sheets
client = gspread.authorize(credentials)

# Replace with your Google Sheet ID

sheet = client.open_by_key(sheet_id).worksheet(sheet_name)

# 3. Read the existing data from the sheet
data = sheet.get_all_values() 

data=pd.DataFrame(data)


In [4]:
data = data.rename(columns={0: 'cmpny',1:'website'})
data['email']=None



In [ ]:
data

In [ ]:
import requests
from bs4 import BeautifulSoup
from queue import Queue, Empty
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.parse import urljoin, urlparse
import pandas as pd
import re
import time
import pandas as pd


#data=pd.read_csv(r'C:\Users\admin\OneDrive\Desktop\work\test.csv')


class EmailScraper:
    def __init__(self, base_url, max_workers=10, queue_size=100):
        self.base_url = base_url
        self.root_url = '{}://{}'.format(urlparse(self.base_url).scheme, urlparse(self.base_url).netloc)
        self.pool = ThreadPoolExecutor(max_workers=max_workers)
        self.scraped_pages = set()
        self.to_crawl = Queue(queue_size)
        self.to_crawl.put(self.base_url)
        self.emails = set()  # Store unique emails

    def parse_links(self, html):
        """Extracts all valid links from the given HTML page and adds them to the crawl queue."""
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all('a', href=True)
        for link in links:
            url = urljoin(self.root_url, link['href'])
            if url.startswith(self.root_url) and url not in self.scraped_pages:
                self.scraped_pages.add(url)
                try:
                    self.to_crawl.put(url, timeout=1)
                except:
                    pass  # Ignore queue full errors

    def extract_emails(self, html):
        """Extracts email addresses from the page content."""
        global data
        emails_found = set(re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", html))
        new_emails = emails_found - self.emails  # Avoid duplicates

        if new_emails:
            self.emails.update(new_emails)
            print(f"[{self.base_url}] Emails found: {new_emails}")

            # Ensure the website is in the DataFrame
            if self.base_url not in data['website'].values:
                data.loc[len(data)] = [self.base_url, str(new_emails)]
            else:
                # Fix: Convert NaN to an empty set before updating
                existing_emails = data.loc[data['website'] == self.base_url, 'email'].values[0]
                
                if pd.isna(existing_emails):  # If NaN, initialize as an empty set
                    existing_emails = set()
                else:
                    existing_emails = eval(existing_emails) if isinstance(existing_emails, str) else existing_emails

                existing_emails.update(new_emails)  # Update with new emails
                data.loc[data['website'] == self.base_url, 'email'] = str(existing_emails)

    def post_scrape_callback(self, res):
        """Processes a webpage once it's downloaded."""
        try:
            result = res.result()
            if result and result.status_code == 200:
                self.parse_links(result.text)  # Extract and queue links
                self.extract_emails(result.text)  # Extract emails
        except Exception as e:
            print(f"Error in callback: {e}")

    def scrape_page(self, url):
        """Fetches a webpage and returns the response."""
        try:
            headers = {"User-Agent": "Mozilla/5.0"}
            res = requests.get(url, headers=headers, timeout=(3, 30))  # Fetch page
            time.sleep(1)  # Be polite to the server
            return res
        except requests.RequestException:
            return None  # Handle network errors gracefully

    def run_scraper(self):
        """Main loop: fetches URLs from the queue and processes them."""
        while True:
            try:
                target_url = self.to_crawl.get(timeout=60)  # Get next URL
                if target_url not in self.scraped_pages:
                    print(f"Scraping URL: {target_url}")
                    self.scraped_pages.add(target_url)
                    job = self.pool.submit(self.scrape_page, target_url)
                    job.add_done_callback(self.post_scrape_callback)
            except Empty:
                print(f"[{self.base_url}] Queue is empty. Exiting...")
                return
            except Exception as e:
                print(f"Unexpected error: {e}")
                continue

def run_multiple_scrapers(urls, max_threads=5):
    """Runs multiple scrapers concurrently for different websites."""
    with ThreadPoolExecutor(max_threads) as executor:
        futures = {executor.submit(EmailScraper(url).run_scraper): url for url in urls}

        for future in as_completed(futures):
            try:
                future.result()  # Ensure all scrapers complete
            except Exception as e:
                print(f"Error in scraper for {futures[future]}: {e}")

if __name__ == '__main__':
    urls = data['website']
    run_multiple_scrapers(urls, max_threads=5)



In [ ]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

# 1. Load your service account credentials
SERVICE_ACCOUNT_FILE = r'c:\Users\admin\Downloads\sreeragapac-2dee079eeff4.json'

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# 2. Connect to Google Sheets
client = gspread.authorize(credentials)

# Replace with your Google Sheet ID

sheet = client.open_by_key(sheet_id).worksheet(sheet_name)

# 3. Read the existing data from the sheet
existing_data = sheet.get_all_values()  # Get all values as a list of lists
existing_row_count = len(existing_data)  # Count the number of existing rows (including the header)

# 4. Load the new data from the CSV file
new_data = data
data=[data]

# 5. Specify the starting row and column for the new data
start_row = existing_row_count + 1  # Start after the last populated row
start_col = 1  # Start in the first column (A)

# 6. Write the new DataFrame to the Google Sheet starting at the specified row
set_with_dataframe(sheet, new_data, row=start_row, col=start_col)

print(f"New data appended successfully starting at row {start_row}!")
final_data = sheet.get_all_values()
print(final_data)#

